In [ ]:
%%capture
!pip install -U classiq

In [ ]:
import classiq
classiq.authenticate()

In [ ]:
from classiq import *

In [ ]:
import itertools
import math, random, inspect
from itertools import product
from typing import List, cast

import numpy as np
from numpy import kron, linalg as LA

# from scipy import linalg

In [ ]:
# To print matrix row in one line
np.set_printoptions(linewidth=None)

### Initial Matrices Preparation

In [ ]:
# Defining all variables
num = 2
N = 2**num
d = 2
k_max = 2
# as the gate counts would increase with t, so we need to define it before creating the circuit
t = 10
# initial_position, initial_velocity are defined below
# Taking matrices M and K based on Theorem 1 of paper

In [ ]:
# As per the Thm. 2, for simplicity
# initial relative positions of N balls
initial_position = np.zeros(N)
# initial_position[0] = 1

# intial velocities of N balls
initial_velocity = np.zeros(N)
initial_velocity[0] = 1;

In [ ]:
# Creating Mass Matrix

# defining a (random) function to get diagonal Mass Matrix elements
def mass_func(i, d):
  return i*i % d + 1 # as mass can't be zero here

# diagonal_elements = [mass_func(i, d) for i in range(1, N + 1)]
diagonal_elements = [1 for i in range(1, N + 1)]

Mass = np.diag(diagonal_elements)

M_sqrt = np.sqrt(Mass)
M_sqrt_inv = np.linalg.pinv(M_sqrt)
# print(Mass)
# print(M_sqrt)
# print(M_sqrt_inv)


Comment on time complexity: there is no meaning of taking matrix M elements using a mass_func, as we still have to iterate  $N=2^n$ times.



Creating K, the Matrix of spring constants

Wrong creation!

In [ ]:
%%script echo Skipping
# let's define each element of K matrix in range (0, k_max) inclusive
# with sparsity d

def create_spring_constants_matrix(N, d, k_max):
    K = np.empty((N, N), dtype=int)
    for i in range(N):
        num_non_zero = random.randint(1, d)
        indices = random.sample(range(N), num_non_zero)
        for j in indices:
            K[i][j] = random.randint(1, k_max+1)
    return K

Correct creation

In [ ]:
# creating K, the Matrix of spring constants
# for simplicity, taking all the spring constants equal to 1 and assuming
# balls are connected linearly and the extremes balls are connected to walls

def create_spring_constants_matrix(N, d, k_max):
  K = np.empty((N, N), dtype=int)
  for i in range(0, N-1):
    K[i][i+1] = K[i+1][i] = 1
  K[0][0] = K[N-1][N-1] = 1
  return K

In [ ]:
K = create_spring_constants_matrix(N, d, k_max)
# print(K)
K_row_sum = [np.sum(row) for row in K]
# print(K_row_sum)

Creating the force matrix $F$

In [ ]:
F = -K
for i in range(N):
  F[i][i] = K_row_sum[i]
# print(F)

Finding matrix $A$ which is given by
$A = \sqrt{M}^{-1} \mathbf{F} \sqrt{M}^{-1}$

In [ ]:
# A = np.dot(np.dot(M_sqrt_inv, F), M_sqrt_inv)
A = np.einsum('ij,jk,kl->il', M_sqrt_inv, F, M_sqrt_inv) # faster for large matrices
# print(A)

From the research paper:

We need to choose $\mathbf{B}$ such that $\vec{\mu}(t)=\mathbf{B}^{\dagger} \vec{y}(t)=$ $\mathbf{B}^{\dagger} \sqrt{\mathbf{M}} \vec{x}(t)$, where $\vec{\mu}(t) \in \mathbb{C}^M$ (for $M=N(N+1) / 2$ ). We can express $\vec{\mu}(t)$ in the basis $\{|j, k\rangle: j \leq k \in[N]\}$, which is of size $M$. Since $\mathbf{F}$ is a graph Laplacian, we can obtain $\mathbf{B}$ from the incidence matrix of $\mathbf{F}$ given by
$$
\sqrt{\mathbf{M}} \mathbf{B} |j, k\rangle=\left\{\begin{array}{ll}
\sqrt{\kappa_{j j}}|j\rangle, & \text { if } j=k \\
\sqrt{\kappa_{j k}}(|j\rangle-|k\rangle), & \text { if } j<k
\end{array} .\right.
$$

This choice satisfies $\sqrt{\mathbf{M}} \mathbf{B}(\sqrt{\mathbf{M}} \mathbf{B})^{\dagger}=\mathbf{F}$, and hence $\mathbf{B B}^{\dagger}=\mathbf{A}$, because we can write $\mathbf{F}=$ $\sqrt{\mathbf{M}}\left(\sum_{j \leq k} \mathbf{B}|j, k\rangle\langle j, k| \mathbf{B}^{\dagger}\right) \sqrt{\mathbf{M}}$.

In [ ]:
m = N*(N+1)//2

#### Skipping

How would be find the columns of matrix $\sqrt{\mathbf{M}} \mathbf{B}$ for $k>N$ ???

We don't have $K_{jk}$ elements for $k>N$ as matrix $K$ is only $N \times N$.

In [ ]:
%%script echo Skipping: could not understand the way to write B and B_dag
def create_matrix_B(N, m, K, M_sqrt_inv):
    B = np.zeros((N, m), dtype=int)
    for j in range(N):
        for k in range(N):
            if j == k:
                B[j][j] = np.sqrt(K[j][j])
            elif j < k:
                B[j][k] = np.sqrt(K[j][k]) * (j - k)
    # Create matrix B
    print("Printing dot product of M_sqrt_B and it's conjugate ")
    B_dagy = B.conj().T
    print(np.dot(B, B_dagy))

    B = np.dot(M_sqrt_inv, B)
    return B

In [ ]:
%%script echo Skipping: could not understand the way to write B and B_dag
print("F is: \n", F)
B = create_matrix_B(N, m, K, M_sqrt_inv)
B_dag = B.conj().T
# print(B)
# print(B_dag)
print(np.dot(B, B_dag))
print(A)


If we have $B$ and $B^\dagger$ matrix, we can find matrix $H$ as following.

In [ ]:
%%script echo Skipping: could not understand the way to write B and B_dag
H = np.zeros((N+m, N+m), dtype=B.dtype)
# Fill in the blocks
H[:N, N:] = -B
H[N:, :N] = -B_dag
print(H)

#### Continue

The matrix $H$ is defined as

$
\mathbf{H}:=-\left(\begin{array}{cc}
\mathbf{0} & \mathbf{B} \\
\mathbf{B}^{\dagger} & \mathbf{0}
\end{array}\right)
$

For $H$ to be Hermitian,

$ H = H^\dagger \iff \begin{bmatrix} 0 & B^\dagger \\ B & 0 \end{bmatrix} = \begin{bmatrix} 0 & B \\ B^\dagger & 0 \end{bmatrix} $


(The dimension of each $\mathbf{0}$ is clear from context.) Here $\mathbf{H}$ acts on the space $\mathbb{C}^{N+M}$ and functions as the "square root" of $\mathbf{A}$, since the first block of $\mathbf{H}^2$ is $\mathbf{A}$ or

$ H^2 = A \left[ \begin{array}{cc} I & 0 \\ 0 & I \end{array} \right]$

### Creating a hermitian matrix $H$ randomly

In [ ]:
def create_matrix_H(N, m, a, b):
  H = np.zeros((N+m, N+m), dtype=complex)
  for i in  range(N, m+N):
    for j in range(0, N):
      H[i][j] = H[j][i] = random.randint(a, b)
  return H

In [ ]:
# let's take values of H in range [a, b], random range
a = 0
b = 3
H = create_matrix_H(N, m, a, b)
H_dag = np.conj(H.T)

print(H)
# print(H_dag)

In [ ]:
# Checking if H is hermitian (:
np.allclose(H, H_dag)

Extracting $B$ and $B^\dagger$ from $H$

In [ ]:
def create_matrix_B_dag(N, m):
  B_dag = np.empty((m, N), dtype=complex)
  for i in range(N, m+N):
    for j in range(0, N):
      B_dag[i-N][j] = H[i][j]
  return B_dag

B_dag = create_matrix_B_dag(N, m)
# print(B_dag)

Making the matrix $H$ equavalent to matrix $M$ of qpe-for-matrix notebook from classiq tutorials.

In [ ]:
const = -t/(2*np.pi)
H = const * H
# print(H)
M = np.matrix(H)

In [ ]:
# eigval_H, eigvec_H = np.linalg.eig(H)
# print(eigval_H)


Schrödinger's equation induced by $\mathbf{H}$ is
$$
|\dot{\psi}(t)\rangle=-i \mathbf{H}|\psi(t)\rangle,
$$
where $|\psi(t)\rangle \in \mathbb{C}^{N+M}$ is the state of a quantum system at time $t$.

It can be verified by direct substitution that
$$
|\psi(t)\rangle \propto\left(\begin{array}{c}
\dot{\vec{y}}(t) \\
i \mathbf{B}^{\dagger} \vec{y}(t)
\end{array}\right) .
$$
is a valid solution to Eq. (13) in the paper. Hence we have
$$
\left(\begin{array}{c}
\dot{\vec{y}}(t) \\
i \mathbf{B}^{\dagger} \vec{y}(t)
\end{array}\right)=e^{-i t \mathbf{H}}\left(\begin{array}{c}
\dot{\vec{y}}(0) \\
i \mathbf{B}^{\dagger} \vec{y}(0)
\end{array}\right),
$$
which lets us compute $\dot{\vec{y}}(t)$ and $\mathbf{B}^{\dagger} \vec{y}(t)$ using Hamiltonian simulation.

### Set the Initial Vector


In [ ]:
pot_y = np.dot(M_sqrt, initial_position)
kin_y = np.dot(M_sqrt, initial_velocity)
B_dag_Y = np.dot(B_dag, pot_y)
int_vec = np.hstack((kin_y, B_dag_Y))
print("Initial state is", int_vec)

## From QPE FOR MATRIX on Classiq Tutorials

### 2.1 Classical Functions

#### 2.1.1 Pauli Decomposition

To translate the matrix into quantum circuit language, write the matrix in the form of a list of strings of composite Pauli operators.

In [ ]:
Paulidict = {
    "I": np.array([[1, 0], [0, 1]], dtype=np.complex128),
    "Z": np.array([[1, 0], [0, -1]], dtype=np.complex128),
    "X": np.array([[0, 1], [1, 0]], dtype=np.complex128),
    "Y": np.array([[0, -1j], [1j, 0]], dtype=np.complex128),
}


# generate all combinations of Pauli strings of size n
def generate_all_pauli_strings(seq, n):
    for s in product(seq, repeat=n):
        yield "".join(s)


# convert a Paulistring of size n to 2**n X 2**n matrix
def pauli_string_2mat(seq):
    myPmat = Paulidict[seq[0]]
    for p in seq[1:]:
        myPmat = kron(myPmat, Paulidict[p])
    return myPmat


# Hilbert-Schmidt-Product of two matrices M1, M2
def hilbert_schmidt(M1, M2):
  M1 = np.matrix(M1)
  M2 = np.matrix(M2)
  return (np.dot(M1.conj().T, M2)).trace()


# naive decomposition, running over all HS products for all Pauli strings
def lcu_naive(H, N, m):
    n = int(np.log2(N+m))
    myPauliList = list(generate_all_pauli_strings("IZXY", n))
    print(myPauliList)
    print(pauli_string_2mat(myPauliList[1]))
    mylist = []

    for pstr in myPauliList:
        co = (1 / 2**n) * hilbert_schmidt(pauli_string_2mat(pstr), H)
        # --> trace of (M1^dag @ M2) divided by 2**n
        if co != 0:
            mylist = mylist + [(pstr, co)]

    return mylist

#### 2.1.2 Parser to the `PauliTerm` struct

A Hamiltonian is defined by a list of PauliTerm structs. Define a function that transforms between the Pauli list representation to this struct presentation.

In [ ]:
from classiq import Pauli, PauliTerm

my_list = {"I": Pauli.I, "X": Pauli.X, "Y": Pauli.Y, "Z": Pauli.Z}


def pauli_str_to_enums(pauli):
    return [my_list[s] for s in pauli]

# ye wala function sidha suzuki trotter ko denge
def pauli_operator_to_hamiltonian(pauli_list): # pauli_list => pauli_ops --> pairs of pauli seq and coefficients
    return [
        PauliTerm(
            pauli=pauli_str_to_enums(pauli), coefficient=cast(complex, coeff).real
        )
        for pauli, coeff in pauli_list
    ]

#### 2.1.3 Matrix Rescaling

As QPE obtains a phase in the form $e^{2\pi i\theta}$, there is meaning only for $\theta \in [0,1)$. However, the matrix M can have any eigenvalue. To fix this discrepancy, the values of the matrix stretch to be rescaled. If
$\theta \in [\lambda_{min}, \lambda_{max}]$ we can use a normalization function to map those values into $[0, 1-1/{2^m}]$, where $m$
is the size of the QPE register.

Perform the normalization procedure by:


In [ ]:
def normalization_params(pauli_list):
    return len(pauli_list[0][0]) * sum(
        [abs(pauli_list[k][1]) for k in range(len(pauli_list))]
    )

b. Defining the function `normalize_hamiltonian` that shifts the matrix by adding $\lambda*I^n$ to the Pauli list. (The evaluated span is thus $\theta\in[0, 2*\lambda]$) and normalizes it by multiplying all the Pauli coefficients by $(1-1/2^n)/(2*\lambda)$ (the evaluated span is then $\theta\in [0, 1-1/2^n]$, as required.)

In [ ]:
# normalization_coeff = normalization_params(pauli_ops) => lambda
# k = n_qpe --> defined under precesion
def normalize_hamiltonian(pauli_list, normalization_coeff, k): # pauli_list here is un-normalized
    list_size = len(pauli_list)
    num_qubits = len(pauli_list[0][0])
    normalization = (1 - 1 / (2**k)) / (2 * normalization_coeff)
    # Normalizing the pauli list
    normalized_list = [
        (pauli_list[k][0], pauli_list[k][1] * normalization) for k in range(list_size)
    ]
    # checking if "II" is in pauli_list or not, if yes then normalizing it in following way
    if "I" * num_qubits in [pauli_list[k][0] for k in range(list_size)]:
        id_index = [y[0] for y in pauli_list].index("I" * num_qubits)
        normalized_list[id_index] = (
            "I" * num_qubits,
            (pauli_list[id_index][1] + normalization_coeff) * normalization,
        )
    else:
        normalized_list.append(("I" * num_qubits, normalization_coeff * normalization))

    return normalized_list

#### 2.1.3 QPE Precision Estimator

For QPE algorithms, the precision is set by phase register size $m$, such that the resolution is $1/{2^m}$. If the matrix needs to be normalized, the resolution will be distorted. In the case of normalization, the span of results for the QPE stretches between the lowest and highest possible phase, thus the resolution is mapped to $normalization-coefficient/{2^m} ~\sim 1/{((\lambda_{max}-\lambda_{min})*2^m)}$.

In [ ]:
# if get_recommended_qpe_size = True then this will run otherwise n_qpe
# desired_resolution = 0.02 --> unprecision(+-) of phase
def get_qpe_precision(pauli_list, desired_resolution):
    nqpe = math.log2(2 * normalization_params(pauli_list) / desired_resolution)
    return math.ceil(nqpe)

## 2.2 Quantum Functions

### 2.2.1 A Flexible QPE

Define a flexible QPE function, which allows you to prescribe the "telescopic" expansion of the powered unitary via the `unitary_with_power` "QCallable" See the High Level Modeling Flexible QPE tutorial for more details.

In [ ]:
from classiq import (
    H,
    Output,
    QArray,
    QBit,
    QCallable,
    QParam,
    apply_to_all,
    bind,
    control,
    invert,
    qft,
    qfunc,
    repeat,
)


@qfunc
# unitary_with_power --> either suzuki_troter1_with_power_logic or unitary_with_power_logic
def my_qpe_flexible(
    unitary_with_power: QCallable[QParam[int], QArray[QBit]],
    precision: QParam[int], # precision is n_qpe here
    state: QArray[QBit],
    phase: Output[QArray[QBit, "precision"]],
    # precisioin --> length of qarray of target qubits, more the precision, more the qubits required
) -> None:
    allocate(precision, phase)
    apply_to_all(H, phase)

    repeat(
        count=precision,
        iteration=lambda index: control(
            operand=lambda: unitary_with_power(2**index, state),
            ctrl=phase[index],
        ),
        # index --> this will run 'precision' times, mtlb index=range(0, precision)
    )

    invert(lambda: qft(phase))

### 2.2.2 A First Order Suzuki Trotter with power-logic
The first-order Suzuki-Trotter decomposition approximates the time evolution operator up to the first order of accuracy in small time step size. It's typically expressed as:

$U(\delta t) \approx e^{-iH_1\delta t/2}e^{-iH_2\delta t/2}\ldots$

$H_1, H_2, $ etc., are components of the full Hamiltonian, and $δt$ is the time step.

From Classiq documentations:

We write the Hamiltonian as a sum of Pauli strings $H=\sum_{i=0}^{L-1} a^{(k)} P^{(k)}$, where $a^{(k)}$ are complex coefficients, and each of $P^{(k)}$ is a Pauli string of the form $s_0 \otimes s_1 \otimes \cdots \otimes s_L$, with $s_i \in\{I, X, Y, Z\}$. Approximating Hamiltonian simulation with order 1 Trotter Suzuki refers to:
$$
e^{2 \pi i H} \approx\left(\Pi_{i=0}^{L-1} e^{\frac{a^{(k)}}{r} P^{(k)}}\right)^r
$$
where $r$ is called the number of repetitions.

Wrapping the Trotter-Suzuki function of order 1 with a "power-logic" for the repetition as a function of its power.

In [ ]:
from classiq import suzuki_trotter
from classiq.qmod.symbolic import ceiling, log


@qfunc
def suzuki_trotter1_with_power_logic(
    hamiltonian: QParam[List[PauliTerm]],
    pw: QParam[int],
    r0: QParam[int],
    reps_scaling_factor: QParam[float],
    evolution_coefficient: QParam[float],
    target: QArray[QBit],
) -> None:
    suzuki_trotter(
        hamiltonian,
        evolution_coefficient=evolution_coefficient * pw,
        order=1,
        repetitions=r0 * ceiling(reps_scaling_factor ** (log(pw, 2))),
        qbv=target,
    )

### 2.2.3 A Unitary with power-logic

As an alternative to the Trotter-Suzuki formula, you can work with an exact unitary decomposition. In this case, the power-logic is naive:

In [ ]:
from classiq import power, unitary


@qfunc
def unitary_with_power_logic(
    pw: QParam[int], matrix: QParam[List[List[float]]], target: QArray[QBit]
) -> None:
    power(pw, lambda: unitary(elements=matrix, target=target))

## 3. Preparing the Matrix for QPE

## Can't go forward this
We need to satisfy the following relation, which is not possible for atleast small integers.

$ N + N(N+1)/2 = N(N+3)/2 = 2^x$

$ 2^n (2^n +3)/2 = 2^x $

This relation is needed so that the matrix returned by `hilbert_schmidt` function in `lcu_naive` function has same dimension as matrix $H or M$


In [ ]:
pauli_ops = lcu_naive(M, N, m)
print(pauli_ops)

In [ ]:
N_qubits = len(pauli_ops[0][0])
print("number of qubits: ", N_qubits)

### 3.1 Choose the Algorithm's Precision

Choose the precision using the `n_qpe` parameter or set your desired resolution. If you choose the resolution and set the parameter `get_recommended_qpe_size` parameter to True, the number of qubits is calculated for you accordingly.

In [ ]:
n_qpe = 8

# recommended QPE_SIZE:
get_recommended_qpe_size = True

desired_resolution = 0.01


if get_recommended_qpe_size:
    n_qpe = get_qpe_precision(pauli_ops, desired_resolution)

print("number of qubits for QPE is", n_qpe)

### 3.2 Normalize the Matrix

Transform the matrix to ensure its eigenvalues are between $0$ to $1-(1/2^m)$. The QPE procedure is performed on the new normalized matrix. After the phases are obtained, gather the original phases of the pre-normalized matrix by performing opposite steps to this normalization procedure.

* If the matrix eigenvalues are naturally between the values $0$ to $1-(1/2^n)$, you may not want to normalize them as that  may enlarge the span, thus lowering the resolution of the algorithm. In this case, skip those lines or change the value of `normalize` to False.

In [ ]:
# normalizing the operator
## create a matrix such that its normalized version has eigenvalues of [0,1/2^k] where k is the resolution of the QPE
normalize = True
if normalize:
    normalization_coeff = normalization_params(pauli_ops)
    new_pauli_list = normalize_hamiltonian(pauli_ops, normalization_coeff, n_qpe)
    print(new_pauli_list)

    size = math.sqrt(M.size)
    I = np.eye(int(size))

    Mnew = (
        (M + normalization_coeff * I) * (1 - 1 / (2**n_qpe)) / (2 * normalization_coeff)
    )

else:
    Mnew = M

## 4. Building the Quantum Model

Create a quantum model of the QPE algorithm using the Classiq platform with your desired constraints and preferences.

There are generally two methods for inserting the matrix into the QFT: unitary implementation, which is exact but long; and exponentiation, which is approximated but shorter in depth. Choose the parameter `IS_EXACT` to indicate the chosen method.

In [ ]:
import scipy

from classiq import Output, QNum, allocate, create_model, if_, prepare_amplitudes

IS_EXACT = True

my_amp = (
    int_vec / np.linalg.norm(int_vec)
).tolist()  # amplitude is given by the eignevector


@qfunc
def main(phase_result: Output[QNum[n_qpe, False, n_qpe]]) -> None:
    state = QArray("state")
    prepare_amplitudes(my_amp, 0.0, state)
    phase_array = QArray("phase_array")
    my_qpe_flexible(
        unitary_with_power=lambda pw, target: if_(
            condition=IS_EXACT,
            then=lambda: unitary_with_power_logic(
                matrix=scipy.linalg.expm(1j * 2 * np.pi * Mnew).tolist(),
                pw=pw,
                target=target,
            ),
            else_=lambda: suzuki_trotter1_with_power_logic(
                hamiltonian=pauli_operator_to_hamiltonian(pauli_ops),
                pw=pw,
                r0=2,
                reps_scaling_factor=1.8,
                evolution_coefficient=-2 * np.pi,
                target=target,
            ),
        ),
        precision=n_qpe,
        state=state,
        phase=phase_array,
    )
    bind(phase_array, phase_result)


qmod = create_model(main)

Set execution preferences:

In [ ]:
from classiq import set_execution_preferences
from classiq.execution import ExecutionPreferences

num_shots = 1000
qmod = set_execution_preferences(qmod, ExecutionPreferences(num_shots=num_shots))

In [ ]:
from classiq import write_qmod

write_qmod(qmod, "qpe_for_matrix", decimal_precision=15)

In [ ]:
from classiq import show, synthesize

qprog = synthesize(qmod)
show(qprog)